In [1]:
#importar as bibliotecas
import pandas as pd

#importar base de dados
emails = pd.read_excel(r"Bases de Dados\Emails.xlsx")
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

In [2]:
#incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')

In [3]:
dicionario_lojas={}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

In [4]:
dia_indicador = vendas['Data'].max()

In [24]:
import pathlib

caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo

    dicionario_lojas[loja].to_excel(local_arquivo)

In [9]:
loja = 'Norte Shopping'
vendas_loja = dicionario_lojas[loja]
vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

# faturamento
faturamento_ano = vendas_loja['Valor Final'].sum()
faturamento_dia = vendas_loja_dia['Valor Final'].sum()

# diversidade de produtos
qtde_produtos_ano = len(vendas_loja['Produto'].unique())
qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

# tiket medio
vendas_loja = vendas_loja.drop(columns=vendas_loja.select_dtypes(include='datetime64').columns)
valor_venda = vendas_loja.groupby('Código Venda').sum()
# ticket_medio_ano = valor_venda['Valor Final'].mean()
valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)
ticket_medio_dia = valor_venda_dia['Valor Final'].mean()



In [10]:
# definição de metas

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtd_produtos_dia = 4
meta_qtd_produtos_ano = 120
meta_ticket_medio_dia = 500
meta_ticket_medio_ano = 500

In [1]:
import yaml
import logging

def load_credentials(filepath):
    try:
        with open(filepath, 'r') as file:
            credentials = yaml.safe_load(file)
            user = credentials['user']
            password = credentials['password']
            return user, password
    except Exception as e:
        logging.error("Failed to load credentials: {}".format(e))
        raise

In [2]:
from imap_tools import MailBox, AND
credentials = load_credentials('docs/credentials.yaml')

meu_email = MailBox("imap.gmail.com").login(*credentials)

nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]


mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

cor_fat_dia = 'green'
cor_fat_ano = 'green'
cor_qtde_dia = 'green'
# cor_qtde_ano = 'green'
# cor_ticket_dia = 'green'
# cor_ticket_ano = 'green'

mail.HTMLBody = f'''
<p>Bom dia, {nome}</p>

<p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja} foi:<?strong></p>

<table>
  <tr>
    <th>Indicador</th>
    <th>Valor Dia</th>
    <th>Meta Dia</th>
    <th>Cenário Dia</th>
  </tr>
  <tr>
    <td>Faturamento</td>
    <td style="text-align: center">{faturamento_dia}</td>
    <td style="text-align: center">{meta_faturamento_dia}</td>
    <td style="text-align: center">X</td>
  </tr>
  <tr>
    <td>Diversidade de Produtos</td>
    <td style="text-align: center">{qtde_produtos_dia}</td>
    <td style="text-align: center">{meta_qtd_produtos_dia}</td>
    <td style="text-align: center">X</td>
  </tr>
  <tr>
    <td>Ticket Médio</td>
    <td style="text-align: center">{ticket_medio_dia}</td>
    <td style="text-align: center">{meta_ticket_medio_dia}</td>
    <td style="text-align: center">X</td>
  </tr>
  
</table>

<p>Segue em anexo a planilha com todos od dados para mais detalhes.</p>

<p>Qualquer dúvida estou à disposição</p>
<p>Att., Neverton</p>
'''

#anexos 
attachment = pathlib.Path.cwd / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
mail.Attachments.Add(str(attachment))

# mail.send()